In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_11.csv",r"B:\project\ai\ai_chunk_11.csv","B:\project\human\human_chunk_12.csv",r"B:\project\ai\ai_chunk_12.csv","B:\project\human\human_chunk_13.csv",r"B:\project\ai\ai_chunk_12.csv","B:\project\human\human_chunk_14.csv",r"B:\project\ai\ai_chunk_11.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)


(118236, 2)


In [4]:
from transformers import LongformerTokenizer
data=df['text'].tolist()
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokens=tokenizer(data,return_tensors='pt',padding=True,truncation=True)

In [5]:
from sklearn.model_selection import train_test_split
x=tokens['input_ids']
y=df['generated']
xtrain,xtemp,ytrain,ytemp=train_test_split(x,y,test_size=0.3,random_state=42)
xtest,xval,ytest,yval=train_test_split(xtemp,ytemp,test_size=0.3,random_state=42)
xtest,xtrain,xval=xtest.numpy(),xtrain.numpy(),xval.numpy()

In [6]:
from keras.models import load_model
model=load_model('finetuned3.keras')
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=5,batch_size=256,validation_data=(xval,yval))



Epoch 1/5


324/324 [==============================] - 234s 708ms/step - loss: 0.0033 - accuracy: 0.9992 - val_loss: 0.0025 - val_accuracy: 0.9995
Epoch 2/5
324/324 [==============================] - 232s 715ms/step - loss: 5.7455e-04 - accuracy: 0.9999 - val_loss: 0.0019 - val_accuracy: 0.9995
Epoch 3/5
324/324 [==============================] - 231s 714ms/step - loss: 8.2909e-04 - accuracy: 0.9999 - val_loss: 0.0047 - val_accuracy: 0.9992
Epoch 4/5
324/324 [==============================] - 231s 712ms/step - loss: 2.4466e-04 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 0.9995
Epoch 5/5
324/324 [==============================] - 230s 711ms/step - loss: 2.2807e-05 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 0.9996


In [7]:
model.evaluate(xtest,ytest)
model.save('finetuned4.keras')

776/776 [==============================] - 15s 19ms/step - loss: 0.0020 - accuracy: 0.9997
